In [102]:
# !pip install sqlite3

In [105]:
import pandas as pd
from tqdm import tqdm

# Создаем базу данных
добавить описание!

In [252]:
import sqlite3

def create_database(db_file):
    """Создает базу данных SQLite с заданной структурой."""

    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Создаем таблицы
    cursor.execute("""
        CREATE TABLE DOCS (
            doc_id INTEGER,
            doc_name TEXT,
            doc_text TEXT,
            gender TEXT
        )
    """)

    cursor.execute("""
        CREATE TABLE WORD (
            word_id INTEGER,
            word_position INTEGER,
            word TEXT,
            lemma TEXT,
            lemma_id INTEGER,
            tag TEXT
        )
    """)

    cursor.execute("""
        CREATE TABLE SENT (
            doc INTEGER,
            sent_id INTEGER,
            sent_position INTEGER,
            sentence_text TEXT
        )
    """)

    cursor.execute("""
        CREATE TABLE INVERTED_WORD_INDEX (
            sent INTEGER,
            word_id INTEGER
        )
    """)

    cursor.execute("""
        CREATE TABLE INVERTED_LEMMA_INDEX (
            sent INTEGER,
            lemma_id INTEGER
        )
    """)

    # Создаем индексы
    cursor.execute("CREATE INDEX idx_sent_id ON SENT (sent_id)")
    cursor.execute("CREATE INDEX idx_sent_doc ON SENT (doc, sent_id)") 
    cursor.execute("CREATE INDEX idx_inverted_word_index ON INVERTED_WORD_INDEX (sent, word_id)") 
    cursor.execute("CREATE INDEX idx_inverted_lemma_index ON INVERTED_LEMMA_INDEX (sent, lemma_id)")
    cursor.execute("CREATE INDEX idx_inverted_word_index_word_id ON INVERTED_WORD_INDEX (word_id)")
    cursor.execute("CREATE INDEX idx_inverted_lemma_index_lemma_id ON INVERTED_LEMMA_INDEX (lemma_id)")
    cursor.execute("CREATE INDEX idx_word_id ON WORD (word_id)")
    cursor.execute("CREATE INDEX idx_lemma_id ON WORD (lemma_id)")

    # Сохраняем изменения и закрываем подключение
    conn.commit()
    conn.close()

if __name__ == "__main__":
    db_file = "sem_notes_pre.db"  # Имя файла базы данных
    create_database(db_file)
    print(f"База данных '{db_file}' создана успешно!")

База данных 'sem_notes_pre.db' создана успешно!


# Обработка данных из большого датафрейма

Общий датафрейм содержит всю информацию, которая нужна, но его элементарная единица -- это словоформа. Поэтому придется убрать дублирование предложений и документов. 

In [242]:
df = pd.read_csv('nlp_project_corpus_df - nlp_project_corpus_df .csv')
df

,gender,doc_id,doc_name,doc_text,sent_id,sentence_text,sent_position,word_id,word,lemma,tag,word_position,inverted_word_index,inverted_lemma_index
0,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,1,"Из-за того, что смыслы не имеют материального ...",1,1,из-за,из-за,PR,1,"[1, 24, 60, 440]","[1, 24, 60, 440]"
1,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,1,"Из-за того, что смыслы не имеют материального ...",1,2,того,то,SPRO,2,"[1, 4, 8, 53, 60, 73, 121, 134, 144, 161, 221,...","[1, 2, 4, 8, 11, 13, 18, 22, 24, 26, 29, 32, 3..."
2,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,1,"Из-за того, что смыслы не имеют материального ...",1,3,что,что,CONJ,3,"[1, 11, 26, 53, 55, 60, 73, 87, 94, 101, 104, ...","[1, 11, 26, 53, 55, 60, 73, 87, 94, 101, 104, ..."
3,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,1,"Из-за того, что смыслы не имеют материального ...",1,4,смыслы,смысл,S,4,"[1, 81, 82, 315]","[1, 10, 18, 34, 81, 82, 129, 130, 151, 165, 16..."
4,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,1,"Из-за того, что смыслы не имеют материального ...",1,5,не,не,PART,5,"[1, 2, 4, 5, 9, 10, 11, 14, 18, 21, 24, 26, 28...","[1, 2, 4, 5, 9, 10, 11, 14, 18, 21, 24, 26, 28..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10017,f,34,Грамматика конструкций.,f 34. Грамматика конструкций. Авторы и их посл...,525,Семантические фраземы:\n1) Идиомы (≈ сращения ...,13,10018,значения,значение,S,61,"[4, 11, 28, 32, 53, 68, 156, 159, 169, 190, 23...","[4, 9, 11, 28, 32, 34, 52, 53, 56, 68, 81, 99,..."
10018,f,34,Грамматика конструкций.,f 34. Грамматика конструкций. Авторы и их посл...,525,Семантические фраземы:\n1) Идиомы (≈ сращения ...,13,10019,или,или,CONJ,62,"[10, 11, 26, 34, 40, 41, 50, 56, 60, 71, 76, 1...","[10, 11, 26, 34, 40, 41, 50, 60, 71, 76, 101, ..."
10019,f,34,Грамматика конструкций.,f 34. Грамматика конструкций. Авторы и их посл...,525,Семантические фраземы:\n1) Идиомы (≈ сращения ...,13,10020,формы,форма,S,63,[525],"[6, 18, 60, 63, 97, 98, 199, 223, 224, 348, 35..."
10020,f,34,Грамматика конструкций.,f 34. Грамматика конструкций. Авторы и их посл...,525,Семантические фраземы:\n1) Идиомы (≈ сращения ...,13,10021,составных,составной,A,64,[525],[525]


### Датафрейм уникальных документов

In [243]:
name = []

df_docs = df.copy(deep=True)

for i in tqdm(range(len(df_docs))):
        #doc_id = i
        gender = df_docs.loc[i, 'gender']
        doc_name = df_docs.loc[i, 'doc_name']
        doc_text = df_docs.loc[i, 'doc_text']
        if doc_name not in name:
                name.append(doc_name)
        else:
                df_docs.drop(i, inplace=True)

df_docs.reset_index(inplace=True)

100%|██████████| 10022/10022 [00:24<00:00, 405.52it/s]


In [244]:
docs_ids = {}
# словарь документов {название : айди}

for i in tqdm(range(len(df_docs))):
    docs_ids[df_docs.loc[i, 'doc_name']]=i

docs_ids

100%|██████████| 31/31 [00:00<00:00, 8578.44it/s]


{'История семантики: основные этапы и имена.': 0,
 'Треугольник Фреге.': 1,
 'Теория референции.': 2,
 'Понятие интенсионала и экстенсионала.': 3,
 'Компонентный анализ.': 4,
 'Фон и фигура.': 5,
 'Траектор и ориентир.': 6,
 'Семантические роли.': 7,
 'Фреймовая семантика.': 8,
 'Синкретическое выражение семантических ролей.': 9,
 'Актанты и сирконстанты.': 10,
 'Семантические и синтаксические актанты.': 11,
 'Наследование актантов.': 12,
 'Компоненты значения.': 13,
 'Понятие сферы действия.': 14,
 'Фигура наблюдателя.': 15,
 'Понятие коннотации.': 16,
 'Отрицательная и положительная поляризация.': 17,
 'Полисемия.': 18,
 'Понятие семантического инварианта.': 19,
 'Основные типы семантических сдвигов.': 20,
 'Р.Якобсон о метафоре и метонимии.': 21,
 'А.Вежбицка о метафоре и сравнении.': 22,
 'Дж.Лакофф о концептуальной метафоре.': 23,
 'Концептуальный блендинг.': 24,
 'Е.В.Падучева о механизмах метафоры и метонимии.': 25,
 'Регулярная метонимия (типы с примерами).': 26,
 'Универсальна

### Датафрейм уникальных предложений

In [245]:
df_sent = df.copy(deep=True)
len(df_sent)

10022

In [246]:
text = []

df_sent = pd.DataFrame()

c = 1

for i in tqdm(range(len(df))):
        #doc_id = i
        sent_text = df.loc[i, 'sentence_text']
        sent_position = df.loc[i, 'sent_position']
        sent_id = df.loc[i, 'sent_id']
        #print(df_sent.loc[i, 'doc_name'])
        doc = int(docs_ids[(df.loc[i, 'doc_name'])])

        if (sent_id, sent_text) not in text:
                text.append((sent_id, sent_text))
                df_sent.loc[c, 'doc'] = doc
                df_sent.loc[c, 'id'] = sent_id
                df_sent.loc[c, 'sentence_text'] = sent_text
                df_sent.loc[c, 'sent_position'] = sent_position
                c+=1
        else:
                pass

df_sent.reset_index(inplace=True)

  0%|          | 0/10022 [00:00<?, ?it/s]

100%|██████████| 10022/10022 [00:03<00:00, 2646.11it/s]


In [247]:
df_sent

,index,doc,id,sentence_text,sent_position
0,1,0.0,1.0,"Из-за того, что смыслы не имеют материального ...",1.0
1,2,0.0,2.0,Поэтому семантика долгое время не была настоящ...,2.0
2,3,0.0,3.0,Перелом случился в XX веке.,3.0
3,4,0.0,4.0,до XX века:\n - описывали грамматику и лексику...,4.0
4,5,0.0,5.0,На уровне синхронии – структурализм: не лексик...,5.0
...,...,...,...,...,...
502,503,30.0,521.0,В.В.,9.0
503,504,30.0,522.0,Виноградов \n(классификация восходит к Ш. Балл...,10.0
504,505,30.0,523.0,"корень зла, принимать меры, одержать победу, р...",11.0
505,506,30.0,524.0,И. А. Мельчук.,12.0


### Леммы

In [248]:
lemmas = []

df_lemmas = df.copy(deep=True)

for i in tqdm(range(len(df_lemmas))):
        #doc_id = i
        lemma = df_lemmas.loc[i, 'lemma']
        #print(df_sent.loc[i, 'doc_name'])
        inv_index = (df_lemmas['inverted_lemma_index'])
        #df_sent.loc[i, 'doc'] = doc
        if lemma not in lemmas:
                lemmas.append(lemma)
        else:
                df_lemmas.drop(i, inplace=True)

df_lemmas.reset_index(inplace=True)

lemmas_ids = {}
# словарь название : айди

for i in range(len(df_lemmas)):
    lemmas_ids[df_lemmas.loc[i, 'lemma']]=(i, df_lemmas.loc[i, 'inverted_lemma_index'])

100%|██████████| 10022/10022 [00:18<00:00, 552.81it/s]


В этом словаре хранятся данные в формате
`{'lemma': (id, [inverted indexes])}`

### Запишем инвертированные индексы лемм

In [249]:
df_inv_lemma = pd.DataFrame()
i = 1
for k, v in tqdm(list(lemmas_ids.items())):
    #print(k)
    #print(v[1].replace('[', '').replace(']', '').split(', '):)
    for ind in v[1].replace('[', '').replace(']', '').split(', '):
        #print(ind)
        #print(k)
        #df_inv_lemma['lemma_id'] = v[0]
        lemma_id = int(lemmas_ids[k][0]+1)
        df_inv_lemma.loc[i, 'lemma_id']=lemma_id
        df_inv_lemma.loc[i, 'inverted_lemma_index']=ind
        i+=1


  0%|          | 0/2547 [00:00<?, ?it/s]

100%|██████████| 2547/2547 [00:42<00:00, 60.00it/s] 


### ... И словоформ

In [250]:
wforms = {}

c = 1
for i in range(len(df)):
        #doc_id = i
        word = df.loc[i, 'word']
        #print(word)
        if word not in wforms.keys(): 
            wforms[word]=(c, df.loc[i, 'inverted_word_index'])
            c+=1
        else:
            pass
        
        #print(df_sent.loc[i, 'doc_name'])
        #inv_index = (df_lemmas['inverted_lemma_index'])
        #df_sent.loc[i, 'doc'] = doc


In [251]:
df_inv_word = pd.DataFrame()
i = 1
for k, v in tqdm(list(wforms.items())):
    #print(k)
    #print(v[1].replace('[', '').replace(']', '').split(', '):)
    for ind in v[1].replace('[', '').replace(']', '').split(', '):
        #print(ind)
        #print(k)
        #df_inv_lemma['lemma_id'] = v[0]
        lemma_id = int(wforms[k][0])
        df_inv_word.loc[i, 'word_id']=lemma_id
        df_inv_word.loc[i, 'inverted_word_index']=ind
        i+=1


100%|██████████| 3927/3927 [01:09<00:00, 56.49it/s] 


# Загружаем данные в базу

### Загрузим данные в таблицу WORD

In [253]:
conn = sqlite3.connect('sem_notes_pre.db')
cursor = conn.cursor()

conn.execute("BEGIN TRANSACTION")

try:
        for i in range(len(df)): # загружаем слова
            word = df.loc[i, 'word']
            word_id = wforms[word][0]
            lemma = df.loc[i, 'lemma']
            lemma_id = lemmas_ids[lemma][0]+1
            tag = df.loc[i, 'tag']
            word_position = int(df.loc[i, 'word_position'])

            cursor.execute(
                """
                INSERT INTO WORD (word_id, word_position, word, lemma, lemma_id, tag) 
                VALUES (?, ?, ?, ?, ?, ?)
                """, 
                (word_id, word_position, word, lemma, lemma_id, tag)
            )

        # Сохраняем изменения
        conn.commit()

        
except Exception as e:
        # Если произошла ошибка, откатываем изменения
        conn.rollback()
        raise e

finally:
        # Закрываем соединение
        conn.close()



### Загрузим данные в таблицу DOCS

In [254]:
conn = sqlite3.connect('sem_notes_pre.db')
cursor = conn.cursor()

conn.execute("BEGIN TRANSACTION")

try:

        for i in range(len(df_docs)): # загружаем документы
            doc_id = i+1
            gender = df_docs.loc[i, 'gender']
            doc_name = df_docs.loc[i, 'doc_name']
            doc_text = df_docs.loc[i, 'doc_text']

            cursor.execute(
                """
                INSERT INTO DOCS (doc_id, doc_name, doc_text, gender) 
                VALUES (?, ?, ?, ?)
                """, 
                (doc_id, doc_name, doc_text, gender)
            )

              
        # Сохраняем изменения
        conn.commit()

        
except Exception as e:
        # Если произошла ошибка, откатываем изменения
        conn.rollback()
        raise e

finally:
        # Закрываем соединение
        conn.close()


### Загрузим данные в таблицу SENT

In [255]:
conn = sqlite3.connect('sem_notes_pre.db')
cursor = conn.cursor()

conn.execute("BEGIN TRANSACTION")

try:
        for i in range(len(df_sent)):
            sent_id = df_sent.loc[i, 'id']
            sent_text = df_sent.loc[i, 'sentence_text']
            sent_position = int(df_sent.loc[i, 'sent_position'])
            doc = int(df_sent.loc[i, 'doc'])+1

            cursor.execute(
                """
                INSERT INTO SENT (doc, sent_id, sent_position, sentence_text) 
                VALUES (?, ?, ?, ?)
                """, 
                (doc, sent_id, sent_position, sent_text)
            )

              
        # Сохраняем изменения
        conn.commit()

        
except Exception as e:
        # Если произошла ошибка, откатываем изменения
        conn.rollback()
        raise e

finally:
        # Закрываем соединение
        conn.close()

### Загрузим данные в таблицу INVERTED_LEMMA_INDEX

In [256]:
conn = sqlite3.connect('sem_notes_pre.db')
cursor = conn.cursor()

conn.execute("BEGIN TRANSACTION")

try:
        for i in (range(len(df_inv_lemma))): # загружаем предложения
            lemma_id = int(df_inv_lemma.loc[i+1, 'lemma_id'])
            inverted_lemma_index = int(df_inv_lemma.loc[i+1, 'inverted_lemma_index'])

            cursor.execute(
                """
                INSERT INTO INVERTED_LEMMA_INDEX (sent, lemma_id) 
                VALUES (?, ?)
                """, 
                (inverted_lemma_index, lemma_id)
            )

              
        # Сохраняем изменения
        conn.commit()

        
except Exception as e:
        # Если произошла ошибка, откатываем изменения
        conn.rollback()
        raise e

finally:
        # Закрываем соединение
        conn.close()

### Загрузим данные в таблицу INVERTED_WORD_INDEX

In [257]:
conn = sqlite3.connect('sem_notes_pre.db')
cursor = conn.cursor()

conn.execute("BEGIN TRANSACTION")

try:
        for i in (range(len(df_inv_word))): # загружаем предложения
            word_id = int(df_inv_word.loc[i+1, 'word_id'])
            inverted_word_index = int(df_inv_word.loc[i+1, 'inverted_word_index'])

            cursor.execute(
                """
                INSERT INTO INVERTED_WORD_INDEX (sent, word_id) 
                VALUES (?, ?)
                """, 
                (inverted_word_index, word_id)
            )

              
        # Сохраняем изменения
        conn.commit()

        
except Exception as e:
        # Если произошла ошибка, откатываем изменения
        conn.rollback()
        raise e

finally:
        # Закрываем соединение
        conn.close()

### И задаем запрос

In [258]:
import sqlite3

def get_sentences_with_lemma(db_file, lemma, tag):
    """Выбирает предложения из таблицы SENT, 
    которые содержат слова с заданной леммой.
    """

    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Запрос с несколькими JOIN
    cursor.execute("""
        SELECT DISTINCT sentence_text 
        FROM SENT 
        JOIN INVERTED_LEMMA_INDEX ON INVERTED_LEMMA_INDEX.sent = SENT.sent_id
        JOIN WORD ON INVERTED_LEMMA_INDEX.lemma_id = WORD.lemma_id
        WHERE WORD.lemma = ? and WORD.tag = ?
    """, (lemma, tag))

    # Получение результата
    result = cursor.fetchall()

    # Закрываем подключение
    conn.close()

    return result

db_file = "sem_notes_pre.db"  # Имя файла базы данных
query_word = 'любовь'
tag = 'S'
sentences = get_sentences_with_lemma(db_file, query_word, tag)

print(f"Предложения, содержащие слово '{query_word, tag}':")
for sentence in sentences:
        print(sentence)

Предложения, содержащие слово '('любовь', 'S')':
('Лакофф и М. Джонсон – проекция одного домена на другой: \n        SOURCE DOMAIN => TARGET DOMAIN\n         * Ж. Фоконье и М. Тернер: метафора – совмещение разных доменов (=ментальных пространств)\nConceptual metaphor – семантическое отображение (mapping) связывает SOURCE DOMAIN => TARGET DOMAIN\nСпор – это война \nЛюбовь – это путешествие\nВ домене-цели сохраняется «когнитивная топология» сферы-источника, т.е.',)
